In [1]:
import sys; sys.path.append("..") 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from src.utility import loadfile, savefile, exists
from src.dataprocessing import *
from src import rnn

## load data

In [2]:
# Define variables
input_var   = ["FT-3061-2", "FT-3061-3", "FT-3061-4", "FT-3062-1"]
output_var  = ["TT-3061-3", "TT-3061-5", "LT-3061-2"]
process_var = input_var + output_var

In [3]:
min_len = 100
continuous_path = './data/3_continuous'

cts_list = []
i = 1
while exists(f"{continuous_path}/cts_{min_len}/dataset {min_len}_{i}.csv"):
    cts_df = loadfile(continuous_path, f"cts_{min_len}/dataset {min_len}_{i}", 'csv')
    cts_list.append(cts_df)
    i += 1

csv file is loaded from ./data/3_continuous/cts_100/dataset 100_1.csv
csv file is loaded from ./data/3_continuous/cts_100/dataset 100_2.csv
csv file is loaded from ./data/3_continuous/cts_100/dataset 100_3.csv
csv file is loaded from ./data/3_continuous/cts_100/dataset 100_4.csv
csv file is loaded from ./data/3_continuous/cts_100/dataset 100_5.csv
csv file is loaded from ./data/3_continuous/cts_100/dataset 100_6.csv
csv file is loaded from ./data/3_continuous/cts_100/dataset 100_7.csv
csv file is loaded from ./data/3_continuous/cts_100/dataset 100_8.csv
csv file is loaded from ./data/3_continuous/cts_100/dataset 100_9.csv
csv file is loaded from ./data/3_continuous/cts_100/dataset 100_10.csv
csv file is loaded from ./data/3_continuous/cts_100/dataset 100_11.csv
csv file is loaded from ./data/3_continuous/cts_100/dataset 100_12.csv
csv file is loaded from ./data/3_continuous/cts_100/dataset 100_13.csv
csv file is loaded from ./data/3_continuous/cts_100/dataset 100_14.csv
csv file is loa

# iteration

In [4]:
target_list = cts_list

# history size and future size
history_size = 40
future_size = 20
step = 1

# variable selection
history_var = process_var
future_var = output_var

history_num = len(history_var)
future_num = len(future_var)

# data to series
history_series = []
future_series = []
for i in range(len(target_list)):
    history, future = data2series(target_list[i], history_size, history_var, future_size, future_var,
                                step, start_idx=0, end_idx=None)
    if not i:
        history_series = history
        future_series = future
    else:
        history_series = np.concatenate([history_series, history], axis=0)
        future_series = np.concatenate([future_series, future], axis=0)
        
# supervised attention factor
delta_list = [0.5, 1, 1.5, 2, 2.5]
att_list = ['linear', 'exp']
        
# test data split        
test_size = 0.2
test_num = -1

# model structure
num_layers = 1
num_neurons = 50
dense_layers = 1
dense_neurons = 50
model_type = 'datt_seq2seq_gru'

for iteration in range(1,11):
    for att_type in att_list:
        for delta in delta_list:
            print(f"{iteration}th iteration")
            print(f"att_type: {att_type}")
            print(f"delta: {delta}")
            factor = rnn.super_attention(delta, future_size, future_num, att_type)
            # Dual-attention Seq2Seq model
            DATT_seq2seq_GRU = rnn.RNN(history_series, history_var, future_series, future_var)
            # TT split
            DATT_seq2seq_GRU.train_test(test_size=test_size, test_num=test_num)
            # TV split
            valid_size = DATT_seq2seq_GRU.history_test.shape[0]/DATT_seq2seq_GRU.history_train.shape[0]
            DATT_seq2seq_GRU.train_valid(valid_size=valid_size)
            # scaling
            DATT_seq2seq_GRU.scaling()
            # modeling
            DATT_seq2seq_GRU.build_model(num_layers=num_layers, num_neurons=num_neurons, dense_layers=dense_layers, dense_neurons=dense_neurons, model_type=model_type, factor=factor)
            # training
            model_num = iteration
            model_name = f"{history_size}_{future_size}_{num_layers}_{num_neurons}_{dense_layers}_{dense_neurons}_{model_type}_{att_type}_{delta}_{model_num}"
            if not exists(f"./model/{model_name}.h5"):
                DATT_seq2seq_GRU.train()
                DATT_seq2seq_GRU.save_model(f"./model/{model_name}", 'weights')
                
            else:
                DATT_seq2seq_GRU.model.load_weights(f"./model/{model_name}.h5")
            # test
            test_result = DATT_seq2seq_GRU.test()
            if not exists(f'./result/{model_name}.csv', ):
                savefile(test_result, './result', model_name)
            print("\n")
print('end opitmization')

1th iteration
att_type: linear
delta: 0.5
Epoch 1/10000
563/563 - 11s - loss: 0.2280 - val_loss: 0.1835 - 11s/epoch - 20ms/step
Epoch 2/10000
563/563 - 5s - loss: 0.1593 - val_loss: 0.1819 - 5s/epoch - 9ms/step
Epoch 3/10000
563/563 - 6s - loss: 0.1485 - val_loss: 0.1574 - 6s/epoch - 11ms/step
Epoch 4/10000
563/563 - 6s - loss: 0.1399 - val_loss: 0.1534 - 6s/epoch - 10ms/step
Epoch 5/10000
563/563 - 6s - loss: 0.1316 - val_loss: 0.1411 - 6s/epoch - 10ms/step
Epoch 6/10000
563/563 - 5s - loss: 0.1251 - val_loss: 0.1371 - 5s/epoch - 9ms/step
Epoch 7/10000
563/563 - 5s - loss: 0.1186 - val_loss: 0.1341 - 5s/epoch - 9ms/step
Epoch 8/10000
563/563 - 5s - loss: 0.1134 - val_loss: 0.1302 - 5s/epoch - 9ms/step
Epoch 9/10000
563/563 - 5s - loss: 0.1084 - val_loss: 0.1200 - 5s/epoch - 8ms/step
Epoch 10/10000
563/563 - 5s - loss: 0.1042 - val_loss: 0.1241 - 5s/epoch - 9ms/step
Epoch 11/10000
563/563 - 5s - loss: 0.0990 - val_loss: 0.1101 - 5s/epoch - 9ms/step
Epoch 12/10000
563/563 - 6s - loss: 0

,TT-3061-3,TT-3061-5,LT-3061-2,mean,TT-3061-3,TT-3061-5,LT-3061-2,mean
index,R2,R2,R2,R2,nRMSE,nRMSE,nRMSE,nRMSE
0,0.846615,0.907208,0.938124,0.897316,7.727636,5.356199,3.820904,5.634913
1,0.796534,0.850468,0.908163,0.851721,8.740353,6.801561,4.656307,6.73274
2,0.707208,0.799141,0.870673,0.792341,10.028142,7.885955,5.527188,7.813761
3,0.610921,0.757385,0.840415,0.73624,11.173961,8.670024,6.141176,8.66172
4,0.515775,0.719383,0.818827,0.684662,12.266004,9.328495,6.544863,9.379787
5,0.421917,0.686782,0.802564,0.637088,13.356922,9.859945,6.833493,10.016787
6,0.332165,0.657959,0.789875,0.593333,14.07488,10.305971,7.051096,10.477316
7,0.251936,0.632061,0.778002,0.553999,14.611558,10.691415,7.248981,10.850651
8,0.182839,0.608312,0.761364,0.517505,15.045828,11.032973,7.517062,11.198621


csv file is saved to: ./result/40_20_1_50_1_50_datt_seq2seq_gru_linear_0.5_1.csv


1th iteration
att_type: linear
delta: 1
Epoch 1/10000
563/563 - 9s - loss: 0.3080 - val_loss: 0.2517 - 9s/epoch - 15ms/step
Epoch 2/10000
563/563 - 5s - loss: 0.2222 - val_loss: 0.2325 - 5s/epoch - 9ms/step
Epoch 3/10000
563/563 - 5s - loss: 0.2056 - val_loss: 0.2195 - 5s/epoch - 8ms/step
Epoch 4/10000
563/563 - 5s - loss: 0.1944 - val_loss: 0.2098 - 5s/epoch - 8ms/step
Epoch 5/10000
563/563 - 5s - loss: 0.1839 - val_loss: 0.2024 - 5s/epoch - 8ms/step
Epoch 6/10000
563/563 - 5s - loss: 0.1758 - val_loss: 0.1919 - 5s/epoch - 8ms/step
Epoch 7/10000
563/563 - 5s - loss: 0.1675 - val_loss: 0.1895 - 5s/epoch - 8ms/step
Epoch 8/10000
563/563 - 5s - loss: 0.1610 - val_loss: 0.1779 - 5s/epoch - 8ms/step
Epoch 9/10000
563/563 - 5s - loss: 0.1561 - val_loss: 0.1716 - 5s/epoch - 8ms/step
Epoch 10/10000
563/563 - 5s - loss: 0.1481 - val_loss: 0.1687 - 5s/epoch - 8ms/step
Epoch 11/10000
563/563 - 5s - loss: 0.1424 - 

,TT-3061-3,TT-3061-5,LT-3061-2,mean,TT-3061-3,TT-3061-5,LT-3061-2,mean
index,R2,R2,R2,R2,nRMSE,nRMSE,nRMSE,nRMSE
0,0.89091,0.883243,0.937504,0.903886,6.516988,6.008184,3.839989,5.455054
1,0.817476,0.843311,0.906571,0.855786,8.278315,6.962429,4.696508,6.64575
2,0.728403,0.790658,0.854301,0.791121,9.658361,8.05076,5.866621,7.858581
3,0.641329,0.761546,0.80798,0.736951,10.728439,8.595364,6.736417,8.68674
4,0.559351,0.736445,0.770419,0.688738,11.701074,9.040461,7.367533,9.369689
5,0.488053,0.713878,0.736677,0.646203,12.569666,9.423814,7.891771,9.96175
6,0.426952,0.692381,0.715451,0.611595,13.037838,9.773632,8.205335,10.338935
7,0.375572,0.672568,0.70534,0.584493,13.3496,10.085735,8.351481,10.595605
8,0.334246,0.652771,0.697157,0.561391,13.580604,10.387974,8.468161,10.812246


csv file is saved to: ./result/40_20_1_50_1_50_datt_seq2seq_gru_linear_1_1.csv


1th iteration
att_type: linear
delta: 1.5
Epoch 1/10000
563/563 - 9s - loss: 0.4277 - val_loss: 0.3429 - 9s/epoch - 16ms/step
Epoch 2/10000
563/563 - 5s - loss: 0.2962 - val_loss: 0.3062 - 5s/epoch - 9ms/step
Epoch 3/10000
563/563 - 5s - loss: 0.2737 - val_loss: 0.3015 - 5s/epoch - 8ms/step
Epoch 4/10000
563/563 - 5s - loss: 0.2565 - val_loss: 0.2752 - 5s/epoch - 8ms/step
Epoch 5/10000
563/563 - 5s - loss: 0.2414 - val_loss: 0.2691 - 5s/epoch - 8ms/step
Epoch 6/10000
563/563 - 5s - loss: 0.2301 - val_loss: 0.2515 - 5s/epoch - 8ms/step
Epoch 7/10000
563/563 - 5s - loss: 0.2206 - val_loss: 0.2402 - 5s/epoch - 8ms/step
Epoch 8/10000
563/563 - 5s - loss: 0.2084 - val_loss: 0.2275 - 5s/epoch - 8ms/step
Epoch 9/10000
563/563 - 5s - loss: 0.1992 - val_loss: 0.2211 - 5s/epoch - 8ms/step
Epoch 10/10000
563/563 - 5s - loss: 0.1916 - val_loss: 0.2117 - 5s/epoch - 8ms/step
Epoch 11/10000
563/563 - 5s - loss: 0.1832 - 

...

csv file is saved to: ./result/40_20_1_50_1_50_datt_seq2seq_gru_exp_1.5_7.csv


7th iteration
att_type: exp
delta: 2
Epoch 1/10000
563/563 - 8s - loss: 0.4571 - val_loss: 0.3489 - 8s/epoch - 14ms/step
Epoch 2/10000
563/563 - 5s - loss: 0.3112 - val_loss: 0.3208 - 5s/epoch - 9ms/step
Epoch 3/10000
563/563 - 5s - loss: 0.2883 - val_loss: 0.2964 - 5s/epoch - 9ms/step
Epoch 4/10000
563/563 - 5s - loss: 0.2751 - val_loss: 0.3019 - 5s/epoch - 9ms/step
Epoch 5/10000
563/563 - 5s - loss: 0.2598 - val_loss: 0.2766 - 5s/epoch - 9ms/step
Epoch 6/10000
563/563 - 5s - loss: 0.2418 - val_loss: 0.2684 - 5s/epoch - 9ms/step
Epoch 7/10000
563/563 - 5s - loss: 0.2346 - val_loss: 0.2531 - 5s/epoch - 9ms/step
Epoch 8/10000
563/563 - 5s - loss: 0.2231 - val_loss: 0.2423 - 5s/epoch - 9ms/step
Epoch 9/10000
563/563 - 5s - loss: 0.2132 - val_loss: 0.2349 - 5s/epoch - 9ms/step
Epoch 10/10000
563/563 - 5s - loss: 0.2053 - val_loss: 0.2368 - 5s/epoch - 9ms/step
Epoch 11/10000
563/563 - 5s - loss: 0.1972 - val_lo

,TT-3061-3,TT-3061-5,LT-3061-2,mean,TT-3061-3,TT-3061-5,LT-3061-2,mean
index,R2,R2,R2,R2,nRMSE,nRMSE,nRMSE,nRMSE
0,0.914075,0.922463,0.939911,0.925483,5.78381,4.896167,3.76534,4.815106
1,0.829878,0.858461,0.89167,0.860003,7.992128,6.61727,5.057161,6.55552
2,0.719532,0.799137,0.837387,0.785352,9.814826,7.88603,6.197808,7.966221
3,0.598027,0.758261,0.790447,0.715579,11.357596,8.654352,7.037246,9.016398
4,0.475449,0.731052,0.755926,0.654142,12.766536,9.132485,7.596531,9.831851
5,0.354513,0.712775,0.729481,0.598923,14.114159,9.441961,7.998869,10.51833
6,0.241907,0.701091,0.708666,0.550554,14.995858,9.634287,8.302596,10.97758
7,0.14465,0.690917,0.688617,0.508061,15.624247,9.799063,8.585197,11.336169
8,0.066062,0.678881,0.662822,0.469255,16.08501,9.989765,8.935317,11.670031


csv file is saved to: ./result/40_20_1_50_1_50_datt_seq2seq_gru_exp_2_7.csv


7th iteration
att_type: exp
delta: 2.5
Epoch 1/10000
563/563 - 8s - loss: 0.5202 - val_loss: 0.4107 - 8s/epoch - 14ms/step
Epoch 2/10000
563/563 - 5s - loss: 0.3691 - val_loss: 0.3771 - 5s/epoch - 9ms/step
Epoch 3/10000
563/563 - 5s - loss: 0.3407 - val_loss: 0.3640 - 5s/epoch - 9ms/step
Epoch 4/10000
563/563 - 5s - loss: 0.3199 - val_loss: 0.3377 - 5s/epoch - 9ms/step
Epoch 5/10000
563/563 - 5s - loss: 0.3005 - val_loss: 0.3161 - 5s/epoch - 9ms/step
Epoch 6/10000
563/563 - 5s - loss: 0.2832 - val_loss: 0.3187 - 5s/epoch - 9ms/step
Epoch 7/10000
563/563 - 5s - loss: 0.2655 - val_loss: 0.2898 - 5s/epoch - 9ms/step
Epoch 8/10000
563/563 - 5s - loss: 0.2523 - val_loss: 0.2776 - 5s/epoch - 9ms/step
Epoch 9/10000
563/563 - 5s - loss: 0.2426 - val_loss: 0.2730 - 5s/epoch - 9ms/step
Epoch 10/10000
563/563 - 5s - loss: 0.2312 - val_loss: 0.2583 - 5s/epoch - 9ms/step
Epoch 11/10000
563/563 - 5s - loss: 0.2216 - val_lo

,TT-3061-3,TT-3061-5,LT-3061-2,mean,TT-3061-3,TT-3061-5,LT-3061-2,mean
index,R2,R2,R2,R2,nRMSE,nRMSE,nRMSE,nRMSE
0,0.893063,0.916504,0.929495,0.913021,6.452371,5.080824,4.078638,5.203944
1,0.771082,0.849602,0.879895,0.833526,9.270918,6.821228,5.324921,7.139022
2,0.648186,0.795285,0.814542,0.752671,10.992516,7.961292,6.618851,8.52422
3,0.52918,0.761502,0.766627,0.685769,12.291806,8.596148,7.426456,9.438137
4,0.417497,0.744644,0.750806,0.637649,13.453284,8.898721,7.675795,10.009266
5,0.315899,0.733328,0.753486,0.600904,14.530201,9.097872,7.635727,10.421267
6,0.228221,0.72078,0.755248,0.568083,15.13062,9.311568,7.609946,10.684045
7,0.152721,0.704849,0.745617,0.534396,15.550351,9.575676,7.759733,10.96192
8,0.086774,0.683322,0.723313,0.497803,15.90565,9.920451,8.094212,11.306771


csv file is saved to: ./result/40_20_1_50_1_50_datt_seq2seq_gru_exp_2.5_7.csv


8th iteration
att_type: linear
delta: 0.5
Epoch 1/10000
563/563 - 8s - loss: 0.2256 - val_loss: 0.1824 - 8s/epoch - 14ms/step
Epoch 2/10000
563/563 - 5s - loss: 0.1605 - val_loss: 0.1651 - 5s/epoch - 9ms/step
Epoch 3/10000
563/563 - 5s - loss: 0.1478 - val_loss: 0.1575 - 5s/epoch - 9ms/step
Epoch 4/10000
563/563 - 5s - loss: 0.1386 - val_loss: 0.1479 - 5s/epoch - 9ms/step
Epoch 5/10000
563/563 - 5s - loss: 0.1313 - val_loss: 0.1393 - 5s/epoch - 9ms/step
Epoch 6/10000
563/563 - 5s - loss: 0.1246 - val_loss: 0.1366 - 5s/epoch - 9ms/step
Epoch 7/10000
563/563 - 5s - loss: 0.1199 - val_loss: 0.1323 - 5s/epoch - 9ms/step
Epoch 8/10000
563/563 - 5s - loss: 0.1139 - val_loss: 0.1270 - 5s/epoch - 9ms/step
Epoch 9/10000
563/563 - 5s - loss: 0.1106 - val_loss: 0.1262 - 5s/epoch - 9ms/step
Epoch 10/10000
563/563 - 5s - loss: 0.1051 - val_loss: 0.1196 - 5s/epoch - 9ms/step
Epoch 11/10000
563/563 - 5s - loss: 0.1012 - v